In [1]:
from finn.util.visualization import showInNetron

from finn.util.test import get_test_model_trained
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

import numpy as np
import qonnx.core.onnx_exec as oxe

import cv2

In [2]:
# ori_onnx = 'BED_classifier__best_mean_F1.onnx'
# clean_onnx = 'BED_classifier__best_mean_F1_clean.onnx'

### 2nd experiments: export to QONNX during training and test with DFire MINI

In [3]:
ori_onnx = 'BED_classifier__best_mean_F1__QONNX.onnx'
clean_onnx = 'BED_classifier__best_mean_F1_clean__QONNX.onnx'

In [4]:
qonnx_cleanup(ori_onnx, out_file=clean_onnx)

In [5]:
showInNetron(ori_onnx)

Serving 'BED_classifier__best_mean_F1__QONNX.onnx' at http://0.0.0.0:8083


In [6]:
showInNetron(clean_onnx)

Stopping http://0.0.0.0:8083
Serving 'BED_classifier__best_mean_F1_clean__QONNX.onnx' at http://0.0.0.0:8083


In [7]:
finn_onnx =  'BED_classifier__best_mean_F1_finn__QONNX.onnx'

In [8]:
model = ModelWrapper(clean_onnx)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save(finn_onnx)

In [9]:
showInNetron(finn_onnx)

Stopping http://0.0.0.0:8083
Serving 'BED_classifier__best_mean_F1_finn__QONNX.onnx' at http://0.0.0.0:8083


# Compare Outputs

In [10]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 224, 224)) 
dummy_in = (dummy_in / 256.).astype(np.float32)

In [11]:
#ori_model = ModelWrapper(ori_onnx)
clean_model = ModelWrapper(clean_onnx)
finn_model = ModelWrapper(finn_onnx)

### Cleaned QONNX

In [12]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[-1.7175992, -0.6809734]], dtype=float32)

### FINN QONNX

In [13]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(finn_model, input_dict)
produced_finn_qonnx = output_dict[list(output_dict.keys())[0]]
produced_finn_qonnx

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


array([[-1.7175992, -0.6809733]], dtype=float32)

# Compare with Images

In [18]:
img_file = './WEB10495.jpg' # Smoke and Fire
#img_file = './WEB10980.jpg' # Only Smoke
#img_file = './WEB10031.jpg' # Empty
img = cv2.imread(img_file)
img = cv2.imread(img_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
img = cv2.resize(img, (224,224), interpolation = cv2.INTER_LINEAR) 
img = img / 256.
img = np.expand_dims(img, axis=0)
img = np.transpose(img, (0, 3, 1, 2))
img = img.astype(np.float32)

In [19]:
print(f'Image shape: {img.shape}')
print(f'Data type: {img.dtype}')

Image shape: (1, 3, 224, 224)
Data type: float32


### Cleaned QONNX

In [20]:
input_dict = {"global_in": img}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[4.4373665, 5.117652 ]], dtype=float32)

### FINN QONNX

In [21]:
input_dict = {"global_in": img}
output_dict = oxe.execute_onnx(finn_model, input_dict)
produced_finn_qonnx = output_dict[list(output_dict.keys())[0]]
produced_finn_qonnx

array([[4.437366, 5.117652]], dtype=float32)